# Introduction
This notebook performs some basic fitting using the example given in Chapter 3 of the [Doing Data Science : Straight Talk from the Frontline book][booklink] by Cathy O'Neil & Rchel Schutt published by O'Reilly Media.

I will be doing the data analysis using Julia.

[booklink]: https://www.oreilly.com/library/view/doing-data-science/9781449363871/

## Linear Regression Exercise
The first exercise is to simulate some data and perform apply Linear Regression to this. The data will be randomly drawn from a Normal distribution with a mean of 5 and a standard deviation of 7. We'll generate 1000 samples.

In [30]:
using Statistics, Random, Gadfly, Plots, GLM, DataFrames
plotly()
Plots.PlotlyBackend()

Plots.PlotlyBackend()

Let's set up the Random Number Generator using the Mersenne Twister algorithm. I'm fixing the rng seed here for repeatability.

In [246]:

rng = MersenneTwister(1234);
x = randn(rng,Float64,1000).*7 .+ 5;

# Lets histogram the data, make sure it looks like we expect.
p1 = histogram(x,bins= :scott)
display(p1)
println(string("Mean = ",mean(x),", stdev = ",std(x)))

<!DOCTYPE html>
 
 
 Plots.jl

Mean = 4.822599208401484, stdev = 7.039220733884908


Now we'll generate some random errors as well as the true relationship between our predictor and outcome variables

In [247]:
true_epsilon = randn(rng,Float64,1000).*2;
true_beta_0 = 1.1;
true_beta_1 = -8.2;
y = true_beta_0 .+ true_beta_1.*x .+ true_epsilon

p1 = histogram(y,bins= :scott)
display(p1)
p2 = Plots.plot(x,y, seriestype = :scatter)
display(p2)
println(string("Mean(y) = ",mean(y),", stdev(y) = ",std(y)))

<!DOCTYPE html>
 
 
 Plots.jl

<!DOCTYPE html>
 
 
 Plots.jl

Mean(y) = -38.38572477374168, stdev(y) = 57.717613863992604


Everything looks pretty good so far, now lets build a regression model and show that it provides good estimates of our beta parameters. We'll use `GLM` to do this, first lets put our x & y into a dataframe.

In [248]:
Data = DataFrame(X = x,Y = y);
model = lm(@formula(Y ~ X),Data)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Y ~ 1 + X

Coefficients:
──────────────────────────────────────────────────────────────────────────
             Estimate  Std. Error  t value  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)   1.13289  0.0770672     14.7     <1e-43   0.981653    1.28412
X            -8.19446  0.00903498  -906.97    <1e-99  -8.21219    -8.17673
──────────────────────────────────────────────────────────────────────────

Look at that, our model gave a pretty good prediction (1.2,-8.2) of our true parameters (1.1, -8.2). Lets print out a couple of metrics

In [249]:
println(string("Standard error of the parameter estimates = ",round.(stderror(model), digits = 5)))
println(string("r^2: ",round.(r2(model), digits = 5)))
println(string("deviance: ",round.(deviance(model), digits = 5)))

Standard error of the parameter estimates = [0.07707, 0.00903]
r^2: 0.99879
deviance: 4032.7385


The next step in the exercise is to generate a new variable `x1` drawn from a gamma distribution. We'll generate the new variable, histogram it, and create a new variable `y1` which is a linear combination of `x` and `x1`

In [250]:
using Distributions
dist = Gamma(2,3);
x1 = rand(dist,1000);
# Lets histogram the data, make sure it looks like we expect.
p1 = histogram(x1,bins= :scott,title ="x1 Histogram",size = (300,200))
display(p1)
# No we'll set y1 to be a linear combination of both x & x1.
true_beta_2 = 7.3;
y1 = true_beta_0 .+ true_beta_1.*x .+ true_beta_2.*x1.+ true_epsilon;
p2 = Plots.plot(x,y1, seriestype = :scatter,size = (400,200),xlabel="x", ylabel = "y1",title="x - y1")
p3 = Plots.plot(x1,y1, seriestype = :scatter,size = (400,200),xlabel="x1", ylabel = "y1",title="x1 - y1")
#display(p2)
#display(p3)
Plots.plot(p2,p3)

<!DOCTYPE html>
 
 
 Plots.jl

<!DOCTYPE html>
 
 
 Plots.jl

We can also plot the data in 3D.

In [251]:
p4 = Plots.plot(x,x1,y1,seriestype = :scatter3d,xlabel="x",ylabel="x1",zlabel = "y1",markersize=0.5)


<!DOCTYPE html>
 
 
 Plots.jl

Now we'll generate 3 models: fit using x only, fit using x1 only, and fit using both. Then we'll look at the fit metrics again.

In [252]:
Data1 = DataFrame(X = x, X1 = x1,Y = y1);
model_x = lm(@formula(Y ~ X),Data1);
model_x1 = lm(@formula(Y ~ X1),Data1);
model_both = lm(@formula(Y ~ X+X1),Data1);
println(string("model_x: Standard error of the parameter estimates = ",round.(stderror(model_x), digits = 5)))
println(string("model_x1: Standard error of the parameter estimates = ",round.(stderror(model_x1), digits = 5)))
println(string("model_both: Standard error of the parameter estimates = ",round.(stderror(model_both), digits = 5)))
println(string("r^2: ",round.(r2(model_x),digits=5), ", ",round.(r2(model_x1),digits=5), ", ",round.(r2(model_both), digits = 5)))
println(string("deviance: ",round.(deviance(model_x), digits = 5),", ",round.(deviance(model_x1), digits = 5), ", ",round.(deviance(model_both), digits = 5)))

model_x: Standard error of the parameter estimates = [1.15668, 0.1356]
model_x1: Standard error of the parameter estimates = [3.22398, 0.44334]
model_both: Standard error of the parameter estimates = [0.12086, 0.00904, 0.01544]
r^2: 0.78742, 0.22132, 0.99906
deviance: 908419.86922, 3.32751956645e6, 4032.65611


There should be no surprise to see that fitting with one variable gives poor results. We're now going to create training and test sets of varying sizes, perform the fits again on these new sets and calculate the training and test MSE. I'll use `Lathe` to split the dataset into test and train.

In [253]:
using Lathe.preprocess: TrainTestSplit
# We'll split by percentage as that is how Lathe requires the input
split_pct = vcat(range(0.01,stop=0.1,length=10), range(0.15,stop=0.5,length=8));
# Create the arrays that will contain the MSE at each iteration for each fit and each subset of data.
mse_train_x = []
mse_train_x1 = []
mse_train_both = []
mse_test_x = []
mse_test_x1 = []
mse_test_both = []
for s in split_pct  
  # Split up the data and generate DataFrams
  test,train = TrainTestSplit(hcat(x,x1,y1),s);
  Data_train = DataFrame(X = train[:,1],Y = train[:,3]);
  Data_test = DataFrame(X = test[:,1],Y = test[:,3]);
  # Fit the model
  model_x = lm(@formula(Y ~ X),Data_train);
  # Get the prediction values from the model for the training and test datasets.
  trainy_pred = predict(model_x,Data_train);
  testy_pred = predict(model_x,Data_test);
  # Append these values to the arrays.
  append!(mse_train_x,mean((Data_train[:,2].-trainy_pred).^2));
  append!(mse_test_x,mean((Data_test[:,2].-testy_pred).^2));
  # Repeat for the other 2 models
  Data_train = DataFrame(X = train[:,2],Y = train[:,3]);
  Data_test = DataFrame(X = test[:,2],Y = test[:,3]);
  model_x1 = lm(@formula(Y ~ X),Data_train);
  trainy_pred = predict(model_x1,Data_train);
  testy_pred = predict(model_x1,Data_test);
  append!(mse_train_x1,mean((Data_train[:,2].-trainy_pred).^2));
  append!(mse_test_x1,mean((Data_test[:,2].-testy_pred).^2));
  Data_train = DataFrame(X = train[:,1],X1 = train[:,2],Y = train[:,3]);
  Data_test = DataFrame(X = test[:,1],X1 = test[:,2],Y = test[:,3]);
  model_both = lm(@formula(Y ~ X+X1),Data_train);
  trainy_pred = predict(model_both,Data_train);
  testy_pred = predict(model_both,Data_test);
  append!(mse_train_both,mean((Data_train[:,3].-trainy_pred).^2));
  append!(mse_test_both,mean((Data_test[:,3].-testy_pred).^2));
end
# Plot the results.
p1 = Plots.plot(split_pct.*1000,mse_train_x,title= "x Fit",label="Train")
Plots.plot!(p1,split_pct.*1000,mse_test_x,size=(300,200),label="Test",xlabel="Test Set Size",ylabel="MSE")
display(p1)
p2 = Plots.plot(split_pct.*1000,mse_train_x1,title="x1 Fit",size=(300,200),label="Train")
Plots.plot!(p2,split_pct.*1000,mse_test_x1,size=(300,200),label="Test",xlabel="Test Set Size",ylabel="MSE")
display(p2)
p3 = Plots.plot(split_pct.*1000,mse_train_both,title="2 Parameter Fit",size=(300,200),label="Train")
Plots.plot!(p3,split_pct.*1000,mse_test_both,size=(300,200),label="Test",xlabel="Test Set Size",ylabel="MSE")
display(p3)


<!DOCTYPE html>
 
 
 Plots.jl

<!DOCTYPE html>
 
 
 Plots.jl

<!DOCTYPE html>
 
 
 Plots.jl

As the number of samples increases in the test set the MSE converges for each fit. This highlights the danger of using a small test set and the necessity of performing cross-validation.

Next I'm going to generate scatter plots for each pair of variables and histograms for each single variable. To do this I will generate a new plotting procedure.

This will consist of 4 x 4 subplots of 4 input variables, the diagonals are histograms of the 4 variables and the subplot in the (i,j) position is the scatter plot of the (i,j) variables. I'd like to generalise this in the future to n variables but for now we'll keep it fixed at 4.

In [278]:
@userplot CompPlots_4x4

@recipe function f(h::CompPlots_4x4)
  if length(h.args) != 5 || !(typeof(h.args[1]) <: AbstractVector) ||
        !(typeof(h.args[2]) <: AbstractVector)||
        !(typeof(h.args[3]) <: AbstractVector)||
        !(typeof(h.args[4]) <: AbstractVector)
        error("CompPlots_4x4 should be given four vectors.  Got: $(typeof(h.args))")
    end
      x1,x2,x3,x4,labs = h.args
      legend := false
    link := :none
    framestyle := [:axes :axes :axes :axes :axes :axes :axes :axes :axes :axes :axes :axes :axes :axes :axes :axes]
    grid := true
      layout := @layout [hist scatter scatter scatter
    scatter hist scatter scatter
    scatter scatter hist scatter
    scatter scatter scatter hist]
      # first histo
    @series begin
        seriestype := :histogram
        subplot := 1
        title := string(labs[1], " Hist")
        x1
    end
      # second histo
    @series begin
        seriestype := :histogram
        subplot := 6
            title := string(labs[2], " Hist")
        x2
    end
      # third histo
    @series begin
        seriestype := :histogram
        subplot := 11
            title := string(labs[3], " Hist")
        x3
    end
      # fourth histo
    @series begin
        seriestype := :histogram
        subplot := 16
            title := string(labs[4], " Hist")
        x4
    end
       # x1-x2 scatter
    @series begin
        seriestype := :scatter
        subplot := 2
        title := string(labs[1]," - ",labs[2])
        x1,x2
    end
      # x1-x3 scatter
    @series begin
        seriestype := :scatter
        subplot := 3
            title := string(labs[1]," - ",labs[3])
        x1,x3
    end
      # x1-x4 scatter
    @series begin
        seriestype := :scatter
        subplot := 4
            title := string(labs[1]," - ",labs[4])
        x1,x4
    end
      
      # x2-x1 scatter
    @series begin
        seriestype := :scatter
        subplot := 5
            title := string(labs[2]," - ",labs[1])
        x2,x1
    end
      # x2-x3 scatter
    @series begin
        seriestype := :scatter
        subplot := 7
                title := string(labs[2]," - ",labs[3])
        x2,x3
    end
      # x2-x4 scatter
    @series begin
        seriestype := :scatter
        subplot := 8
                title := string(labs[2]," - ",labs[4])
        x2,x4
    end
      
      # x3-x1 scatter
    @series begin
        seriestype := :scatter
        subplot := 9
                    title := string(labs[3]," - ",labs[1])
        x3,x1
    end
      # x3-x2 scatter
    @series begin
        seriestype := :scatter
        subplot := 10
                    title := string(labs[3]," - ",labs[2])
        x3,x2
    end
      # x3-x4 scatter
    @series begin
        seriestype := :scatter
        subplot := 12
                    title := string(labs[3]," - ",labs[4])
        x3,x4
    end
      
      # x4-x1 scatter
    @series begin
        seriestype := :scatter
        subplot := 13   
    title := string(labs[4]," - ",labs[1])
        x4,x1
    end
      # x4-x2 scatter
    @series begin
        seriestype := :scatter
        subplot := 14
        title := string(labs[4]," - ",labs[2])
        x4,x2
    end
      # x4-x3 scatter
    @series begin
        seriestype := :scatter
        subplot := 15
        title := string(labs[4]," - ",labs[3])
        x4,x3
    end
end

Lets include a new variable, z = x^2 and include this in the model. Then we'll use the new plotting function I just wrote.

In [279]:
z = x.^2;
true_beta_3 = 2.7;
y2 = true_beta_0 .+ true_beta_1.*x .+ true_beta_2.*x1.+ true_beta_3.*z .+true_epsilon;


In [280]:
# Now we can plot the relationship between all variables
labs = ["x","x1","z","y2"]
p = compplots_4x4(x,x1,z,y2,labs)

<!DOCTYPE html>
 
 
 Plots.jl

The plots are not bad, but could perhaps use some tweaking in the future. Let's perform fits on the new variable, again trying different combinations of the variables to examine the goodness of fit in each case.

In [285]:
Data2 = DataFrame(X = x, X1 = x1,X2 = z,Y = y2);
model_x = lm(@formula(Y ~ X),Data2);
model_x1 = lm(@formula(Y ~ X1),Data2);
model_x2 = lm(@formula(Y ~ X2),Data2);
model_x_x1 = lm(@formula(Y ~ X+X1),Data2);
model_x1_x2 = lm(@formula(Y ~ X1+X2),Data2);
model_x_x2 = lm(@formula(Y ~ X+X2),Data2);
model_all = lm(@formula(Y ~ X+X1+X2),Data2);
println(string("model_x: Parameter estimates = ",round.(coef(model_x), digits = 2)))
println(string("model_x1: Parameter estimates = ",round.(coef(model_x1), digits = 2)))
println(string("model_x2: Parameter estimates = ",round.(coef(model_x2), digits = 2)))
println(string("model_x_x1: Parameter estimates = ",round.(coef(model_x_x1), digits = 2)))
println(string("model_x_x2: Parameter estimates = ",round.(coef(model_x_x2), digits = 2)))
println(string("model_x1_x2: Parameter estimates = ",round.(coef(model_x1_x2), digits = 2)))
println(string("model_all: Parameter estimates = ",round.(coef(model_all), digits = 2)))
println(string("model_x: Standard error of the parameter estimates = ",round.(stderror(model_x), digits = 5)))
println(string("model_x1: Standard error of the parameter estimates = ",round.(stderror(model_x1), digits = 5)))
println(string("model_x2: Standard error of the parameter estimates = ",round.(stderror(model_x2), digits = 5)))
println(string("model_x_x1: Standard error of the parameter estimates = ",round.(stderror(model_x_x1), digits = 5)))
println(string("model_x_x2: Standard error of the parameter estimates = ",round.(stderror(model_x_x2), digits = 5)))
println(string("model_x1_x2: Standard error of the parameter estimates = ",round.(stderror(model_x1_x2), digits = 5)))
println(string("model_all: Standard error of the parameter estimates = ",round.(stderror(model_all), digits = 5)))
println(string("r^2: ",round.(r2(model_x),digits=2), ", ",round.(r2(model_x1),digits=2), ", ",round.(r2(model_x2), digits = 2), ", ",round.(r2(model_x_x1),digits=2), ", ",round.(r2(model_x_x2),digits=2), ", ",round.(r2(model_x1_x2),digits=2), ", ",round.(r2(model_all),digits=2)))
println(string("deviance: ",round.(deviance(model_x),digits=0), ", ",round.(deviance(model_x1),digits=0), ", ",round.(deviance(model_x2), digits = 0), ", ",round.(deviance(model_x_x1),digits=0), ", ",round.(deviance(model_x_x2),digits=0), ", ",round.(deviance(model_x1_x2),digits=0), ", ",round.(deviance(model_all),digits=0)))

model_x: Parameter estimates = [105.12, 20.05]
model_x1: Parameter estimates = [168.18, 5.61]
model_x2: Parameter estimates = [35.94, 2.28]
model_x_x1: Parameter estimates = [68.86, 20.09, 6.02]
model_x_x2: Parameter estimates = [45.38, -8.13, 2.69]
model_x1_x2: Parameter estimates = [-7.7, 7.18, 2.29]
model_all: Parameter estimates = [1.1, -8.2, 7.3, 2.7]
model_x: Standard error of the parameter estimates = [7.29466, 0.85519]
model_x1: Standard error of the parameter estimates = [13.16637, 1.81057]
model_x2: Standard error of the parameter estimates = [1.92867, 0.01544]
model_x_x1: Standard error of the parameter estimates = [11.3419, 0.84838, 1.44925]
model_x_x2: Standard error of the parameter estimates = [1.19607, 0.19728, 0.01367]
model_x1_x2: Standard error of the parameter estimates = [2.41709, 0.30558, 0.0124]
model_all: Standard error of the parameter estimates = [0.12301, 0.01315, 0.01545, 0.00091]
r^2: 0.36, 0.01, 0.96, 0.37, 0.98, 0.97, 1.0
deviance: 3.6130327e7, 5.5496629e

It's clear that the x^2 term is dominating and a linear fit y2 = m*z+c gives a pretty good approximation (r^2 = 0.96). We'll repeat the test we performed earlier with splitting the set into test and train for a couple of the fits.

In [289]:
using Lathe.preprocess: TrainTestSplit
# We'll split by percentage as that is how Lathe requires the input
split_pct = vcat(range(0.01,stop=0.1,length=10), range(0.15,stop=0.5,length=8));
# Create the arrays that will contain the MSE at each iteration for each fit and each subset of data.
mse_train_x2 = []
mse_train_all = []
mse_test_x2 = []
mse_test_all = []
for s in split_pct  
  # Split up the data and generate DataFrams
  test,train = TrainTestSplit(hcat(x,x1,z,y2),s);
  Data_train = DataFrame(X = train[:,3],Y = train[:,4]);
  Data_test = DataFrame(X = test[:,3],Y = test[:,4]);
  # Fit the model
  model_x2 = lm(@formula(Y ~ X),Data_train);
  # Get the prediction values from the model for the training and test datasets.
  trainy_pred = predict(model_x2,Data_train);
  testy_pred = predict(model_x2,Data_test);
  # Append these values to the arrays.
  append!(mse_train_x2,mean((Data_train[:,2].-trainy_pred).^2));
  append!(mse_test_x2,mean((Data_test[:,2].-testy_pred).^2));
  # Repeat for the other model 
  Data_train = DataFrame(X = train[:,1],X1 = train[:,2],X2 = train[:,3],Y = train[:,4]);
  Data_test = DataFrame(X = test[:,1],X1 = test[:,2],X2 = test[:,3],Y = test[:,4]);
  model_all = lm(@formula(Y ~ X+X1+X2),Data_train);
  trainy_pred = predict(model_all,Data_train);
  testy_pred = predict(model_all,Data_test);
  append!(mse_train_all,mean((Data_train[:,4].-trainy_pred).^2));
  append!(mse_test_all,mean((Data_test[:,4].-testy_pred).^2));
end
# Plot the results.
p1 = Plots.plot(split_pct.*1000,mse_train_x2,title= "x^2 Fit",label="Train")
Plots.plot!(p1,split_pct.*1000,mse_test_x2,size=(300,200),label="Test",xlabel="Test Set Size",ylabel="MSE")
display(p1)
p3 = Plots.plot(split_pct.*1000,mse_train_all,title="3 Parameter Fit",size=(300,200),label="Train")
Plots.plot!(p3,split_pct.*1000,mse_test_both,size=(300,200),label="Test",xlabel="Test Set Size",ylabel="MSE")
display(p3)

<!DOCTYPE html>
 
 
 Plots.jl

<!DOCTYPE html>
 
 
 Plots.jl